# README

In [1]:
from dotenv import load_dotenv
import os
import requests
from tqdm.notebook import tqdm as tqdm_notebook

In [2]:
load_dotenv()

BASE_URL = "https://api.github.com"

GITHUB_API_KEY = os.getenv("GITHUB_API_KEY")
headers = {
    "Authorization": f"Bearer {GITHUB_API_KEY}",
    "Accept": "application/vnd.github+json"
}

In [3]:
repos_url = f"{BASE_URL}/users/hieudoanm/repos"

languages = {}
projects = []
total = 0

response = requests.get(repos_url, timeout=10, headers=headers)
if response.status_code == 200:
    repositories = response.json()
    for index, repo in tqdm_notebook(enumerate(repositories)):
        name = repo.get("name", "")
        full_name = repo.get("full_name", "")
        html_url = repo.get("html_url", "")
        languages_url = repo.get("languages_url", "")
        project = str(index + 1) + ". [" + name + "](" + html_url + ")"
        projects.append(project)
        languages_url = f"{BASE_URL}/repos/" + full_name + "/languages"
        languages_response = requests.get(languages_url, timeout=10, headers=headers)
        if languages_response.status_code == 200:
            languages_in_repository = languages_response.json()
            language_keys = list(languages_in_repository.keys())
            for language_key in language_keys:
                percentage = languages_in_repository[language_key]
                if language_key not in languages:
                    languages[language_key] = percentage
                else:
                    languages[language_key] += percentage
                total += int(languages_in_repository[language_key])

languages_percentage = []
for key, value in languages.items():
    languages_percentage.append({
        "language": key,
        "percentage": value / total * 100,
    })
languages_percentage = list(filter(lambda item: item.get("percentage") >= 1,sorted(
    languages_percentage, key=lambda d: d.get("percentage"))))
languages_percentage.reverse()

base_readme = open("./README.base.md", "r", encoding="utf-8")
base_content = base_readme.read()

projects_content = "\n".join(projects)
languages_content = "\n".join(
    map(lambda language_percentage: "- " +
        "`" + language_percentage["language"] + "` - " +
        str(round(language_percentage["percentage"], 2)) + "%",
        languages_percentage)
)
content = base_content.replace("[PROJECTS_PLACEHOLDER]", projects_content).replace(
    "[LANGUAGES_PLACEHOLDER]", languages_content)

readme = open("./README.md", "w", encoding='utf-8')
readme.write(content)
readme.close()

0it [00:00, ?it/s]